本章将聚焦于 Dask 机器学习，主要介绍 Dask-ML 等库的使用。Dask-ML 基于 Dask 的分布式计算能力，面向机器学习应用，可以无缝对接 scikit-learn、XGBoost 等机器学习库。相比之下，Dask-ML 更适合传统机器学习的训练和推理，比如回归、决策树等等，深度学习相关的训练和推理更多基于 PyTorch 或 TensorFlow 等框架。

总结起来，Dask 和 Dask-ML 适合的场景有以下几类：

原始数据无法放到单机内存中，需要进行分布式数据预处理和特征工程；

训练数据和模型可放到单机内存中，超参数调优需要多机并行；

训练数据无法放到单机内存中，需要进行分布式训练。

一方面，Dask 社区将主要精力投入在 Dask DataFrame 上，对 Dask-ML 和分布式训练的优化并不多；另一方面，深度学习已经冲击传统机器学习算法，Dask 设计之初并不是面向深度学习的。读者阅读本章，了解 Dask 机器学习能力后，可以根据自身需求选择适合自己的框架。

# 数据预处理

数据科学工作的重点是理解数据和处理数据，Dask 可以将很多单机的任务横向扩展到集群上，并且可以和 Python 社区数据可视化等库结合，完成探索性数据分析。

分布式数据预处理部分更多依赖 Dask DataFrame 和 Dask Array 的能力，这里不再赘述。

特征工程部分，Dask-ML 实现了很多 sklearn.preprocessing 的 API，比如 MinMaxScaler。对 Dask 而言，稍有不同的是其独热编码，本书写作时，Dask 使用 DummyEncoder 对类别特征进行独热编码，DummyEncoder 是 scikit-learn OneHotEncoder 的 Dask 替代。

# 超参数调优

我们可以使用 Dask 进行超参数调优，主要有两种方式：

- 基于 scikit-learn 的 joblib 后端，将多个超参数调优任务分布到 Dask 集群

- 使用 Dask-ML 提供的超参数调优 API

这两种方式都是针对训练数据量可放到单机内存中的场景。

## scikit-learn joblib

单机的 scikit-learn 已经提供了丰富易用的模型训练和超参数调优接口，它默认使用 joblib 在单机多核之间并行。像随机搜索和网格搜索等超参数调优任务容易并行，任务之间没有依赖关系，很容易并行起来。

### 案例：飞机延误预测（scikit-learn）

下面展示一个基于 scikit-learn 的机器学习分类案例，我们使用 scikit-learn 提供的网格搜索。

In [6]:
import os
import numpy as np
import pandas as pd

file_path = os.path.join("../../../../data/20.others/nyc_flights/", "nyc-flights", "1991.csv")

In [7]:
input_cols = [
    "Year",
    "Month",
    "DayofMonth",
    "DayOfWeek",
    "CRSDepTime",
    "CRSArrTime",
    "UniqueCarrier",
    "FlightNum",
    "ActualElapsedTime",
    "Origin",
    "Dest",
    "Distance",
    "Diverted",
    "ArrDelay",
]

df = pd.read_csv(file_path, usecols=input_cols)
df = df.dropna()

# 预测是否延误
df["ArrDelayBinary"] = 1.0 * (df["ArrDelay"] > 10)

df = df[df.columns.difference(["ArrDelay"])]

# 将 Dest/Origin/UniqueCarrier 等字段转化为 category 类型
for col in df.select_dtypes(["object"]).columns:
    df[col] = df[col].astype("category").cat.codes.astype(np.int32)

for col in df.columns:
    df[col] = df[col].astype(np.float32)

In [8]:
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import GridSearchCV as SkGridSearchCV
from sklearn.model_selection import train_test_split as sk_train_test_split

_y_label = "ArrDelayBinary"
X_train, X_test, y_train, y_test = sk_train_test_split(
    df.loc[:, df.columns != _y_label], 
    df[_y_label], 
    test_size=0.25,
    shuffle=False,
)

model = SGDClassifier(penalty='elasticnet', max_iter=1_000, warm_start=True, loss='log_loss')
params = {'alpha': np.logspace(-4, 1, num=81)}

sk_grid_search = SkGridSearchCV(model, params)

在进行超参数搜索时，只需要添加 `with joblib.parallel_backend('dask'):`，将网格搜索计算任务扩展到 Dask 集群。

In [9]:
import joblib
from dask.distributed import Client, LocalCluster

cluster = LocalCluster()
client = Client(cluster)

In [11]:
with joblib.parallel_backend('dask'):
    sk_grid_search.fit(X_train, y_train)

In [12]:
sk_grid_search.score(X_test, y_test)

0.8082122790726955

## Dask-ML API

### 案例：飞机延误预测（Dask-ML）

Dask-ML 自己也实现了一些超参数调优的 API，除了提供和 scikit-learn 对标的 GridSearchCV、RandomizedSearchCV 等算法外，还提供了连续减半算法、Hyperband 算法等，比如 SuccessiveHalvingSearchCV、HyperbandSearchCV。

下面展示一个基于 Dask-ML 的 Hyperband 超参数调优案例。

Dask-ML 的超参数调优算法要求输入为 Dask DataFrame 或 Dask Array 等可被切分的数据，而非 pandas DataFrame，因此数据预处理部分需要改为 Dask。

值得注意的是，Dask-ML 提供的 SuccessiveHalvingSearchCV 和 HyperbandSearchCV 等算法要求模型必须支持 partial_fit() 和 score()。partial_fit() 是 scikit-learn 中迭代式算法（比如梯度下降法）的一次迭代过程。连续减半算法和 Hyperband 算法先分配一些算力额度，不是完成试验的所有迭代，而只做一定次数的迭代（对 partial_fit() 调用有限次数），评估性能（在验证集上调用 score() 方法），淘汰性能较差的试验。

In [13]:
import dask.dataframe as dd

input_cols = [
    "Year",
    "Month",
    "DayofMonth",
    "DayOfWeek",
    "CRSDepTime",
    "CRSArrTime",
    "UniqueCarrier",
    "FlightNum",
    "ActualElapsedTime",
    "Origin",
    "Dest",
    "Distance",
    "Diverted",
    "ArrDelay",
]

ddf = dd.read_csv(file_path, usecols=input_cols,)

# 预测是否延误
ddf["ArrDelayBinary"] = 1.0 * (ddf["ArrDelay"] > 10)

ddf = ddf[ddf.columns.difference(["ArrDelay"])]
ddf = ddf.dropna()
ddf = ddf.repartition(npartitions=8)

另外，Dask 处理类型变量时与 pandas/scikit-learn 也稍有不同，我们需要：

- 将该特征转换为 category 类型，比如，使用 Dask DataFrame 的 categorize() 方法，或 Dask-ML 的 Categorizer 预处理器。

- 进行独热编码：Dask-ML 中的 DummyEncoder 对类别特征进行独热编码，是 scikit-learn OneHotEncoder 的 Dask 替代。

In [15]:
from dask_ml.preprocessing import DummyEncoder

dummy = DummyEncoder()
ddf = ddf.categorize(columns=["Dest", "Origin", "UniqueCarrier"])
dummified_ddf = dummy.fit_transform(ddf)

并使用 Dask-ML 的 train_test_split 方法切分训练集和测试集：

In [16]:
from dask_ml.model_selection import train_test_split as dsk_train_test_split

_y_label = "ArrDelayBinary"
X_train, X_test, y_train, y_test = dsk_train_test_split(
    dummified_ddf.loc[:, dummified_ddf.columns != _y_label], 
    dummified_ddf[_y_label], 
    test_size=0.25,
    shuffle=False,
)

定义模型和搜索空间的方式与 scikit-learn 类似，然后调用 Dask-ML 的 HyperbandSearchCV 进行超参数调优。

In [17]:
from dask_ml.model_selection import HyperbandSearchCV

# client = Client(LocalCluster())
model = SGDClassifier(penalty='elasticnet', max_iter=1_000, warm_start=True, loss='log_loss')
params = {'alpha': np.logspace(-4, 1, num=30)}

dsk_hyperband = HyperbandSearchCV(model, params, max_iter=243)
dsk_hyperband.fit(X_train, y_train, classes=[0.0, 1.0])

D:\Anaconda\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 30 is smaller than n_iter=81. Running 30 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 30 is smaller than n_iter=34. Running 30 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


HyperbandSearchCV(estimator=SGDClassifier(loss='log_loss', penalty='elasticnet',
                                          warm_start=True),
                  max_iter=243,
                  parameters={'alpha': array([1.00000000e-04, 1.48735211e-04, 2.21221629e-04, 3.29034456e-04,
       4.89390092e-04, 7.27895384e-04, 1.08263673e-03, 1.61026203e-03,
       2.39502662e-03, 3.56224789e-03, 5.29831691e-03, 7.88046282e-03,
       1.17210230e-02, 1.74332882e-02, 2.59294380e-02, 3.85662042e-02,
       5.73615251e-02, 8.53167852e-02, 1.26896100e-01, 1.88739182e-01,
       2.80721620e-01, 4.17531894e-01, 6.21016942e-01, 9.23670857e-01,
       1.37382380e+00, 2.04335972e+00, 3.03919538e+00, 4.52035366e+00,
       6.72335754e+00, 1.00000000e+01])})

In [18]:
dsk_hyperband.score(X_test, y_test)

D:\Anaconda\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but SGDClassifier was fitted without feature names
  warnings.warn(


0.8248179783780376

# 分布式机器学习

如果训练数据量很大，Dask-ML 提供了分布式机器学习功能，可以在集群上对大数据进行训练。目前，Dask 提供了两类分布式机器学习 API：

- scikit-learn 式：与 scikit-learn 的调用方式类似

- XGBoost 和 LightGBM 决策树式：与 XGBoost 和 LightGBM 的调用方式类似

## scikit-learn API

基于 Dask Array、Dask DataFrame 和 Dask Delayed 提供的分布式计算能力，参考 scikit-learn，Dask-ML 对机器学习算法做了分布式的实现，比如 dask_ml.linear_model 中的线性回归 LinearRegression、逻辑回归 LogisticRegression，dask_ml.cluster 中的 KMeans。Dask-ML 尽量保持这些机器学习算法的使用方法与 scikit-learn 一致。

In [19]:
import time
import seaborn as sns
import pandas as pd

import dask_ml.datasets
import sklearn.linear_model
import dask_ml.linear_model
from dask_ml.model_selection import train_test_split

In [20]:
X, y = dask_ml.datasets.make_classification(n_samples=10000, 
        n_features=50, 
        random_state=42,
        chunks=10000 // 100
)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X

D:\Anaconda\Lib\site-packages\dask\base.py:1366: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(


dask.array<normal, shape=(10000, 50), dtype=float64, chunksize=(100, 50), chunktype=numpy.ndarray>

调用 fit() 方法（与 scikit-learn 类似）：

In [21]:
lr = dask_ml.linear_model.LogisticRegression(solver="lbfgs").fit(X_train, y_train)

D:\Anaconda\Lib\site-packages\dask\config.py:693: UserWarning: Configuration key "fuse_ave_width" has been deprecated. Please use "optimization.fuse.ave-width" instead
  warnings.warn(


训练好的模型可以用来预测（predict()），也可以计算准确度（score()）。

In [22]:
y_predicted = lr.predict(X_test)
y_predicted[:5].compute()

array([ True, False, False, False,  True])

In [23]:
lr.score(X_test, y_test).compute()

0.705

如果在单机的 scikit-learn 上使用同样大小的数据训练模型，会因为内存不足而报错。

尽管 Dask-ML 这种分布式训练的 API 与 scikit-learn 极其相似，scikit-learn 只能使用单核，Dask-ML 可以使用多核甚至集群，但并不意味着所有场景下都选择 Dask-ML，因为有些时候 Dask-ML 并非性能或性价比最优的选择。这一点与 Dask DataFrame 和 pandas 关系一样，如果数据量能放进单机内存，原生的 pandas 、NumPy 和 scikit-learn 的性能和兼容性总是最优的。

下面的代码对不同规模的训练数据进行了性能分析，在单机多核且数据量较小的场景，Dask-ML 的性能并不比 scikit-learn 更快。原因有很多，包括：

很多机器学习算法是迭代式的，scikit-learn 中，迭代式算法使用 Python 原生 for 循环来实现；Dask-ML 参考了这种 for 循环，但对于 Dask 的 Task Graph 来说，for 循环会使得 Task Graph 很臃肿，执行效率并不是很高。

分布式实现需要在不同进程间分发和收集数据，相比单机单进程，额外增加了很多数据同步和通信开销。

训练数据量和模型性能之间的关系可以通过学习曲线（Learning Curves）来可视化，随着训练数据量增加，像朴素贝叶斯等算法的性能提升十分有限。如果一些机器学习算法无法进行分布式训练或分布式训练成本很高，可以考虑对训练数据采样，数据大小能够放进单机内存，使用 scikit-learn 这种单机框架训练。

综上，如果有一个超出单机内存的训练数据，要根据问题特点、所使用的算法和成本等多方面因素来决定使用何种方式处理。

## XGBoost 和 LightGBM

XGBoost 和 LightGBM 是两种决策树模型的实现，他们本身就对分布式训练友好，且集成了 Dask 的分布式能力。下面以 XGBoost 为例，介绍 XGBoost 如何基于 Dask 实现分布式训练，LightGBM 与之类似。

在 XGBoost 中，训练一个模型既可以使用 train() 方法，也可以使用 scikit-learn 式的 fit() 方法。这两种方式都支持 Dask 分布式训练。

下面的代码对单机的 XGBoost 和 Dask 分布式训练两种方式进行了性能对比。如果使用 Dask，用户需要将 xgboost.DMatrix 修改为 xgboost.dask.DaskDMatrix，xgboost.dask.DaskDMatrix 可以将分布式的 Dask Array 或 Dask DataFrame 转化为 XGBoost 所需要的数据格式；用户还需要将 xgboost.train() 修改为 `xgboost.dask.train()`；并传入 Dask 集群客户端 client。

如果是 XGBoost 的 scikit-learn 式 API，需要将 xgboost.XGBClassifier 修改为 `xgboost.dask.DaskXGBClassifier` 或者 xgboost.XGBRegressor 修改为 `xgboost.dask.DaskXGBRegressor`。